In [ ]:
%cd /home/ibmelab/Documents/GG/VSLRecognition/AUTSL/AAGCN/
import os
import csv
import re

#folder_path = r'dataset'  # Use raw string to avoid issues with backslashes
#folder_path = r'30 class 28 actor (center)'
folder_path = r'/home/ibmelab/Documents/GG/VSLRecognition/AUTSL/videos'
csv_file_path = 'videos_list.csv'
labels_file_path = '1_1000_label.csv'
final_file_path = 'temp_videos_list.csv'

# Read the label and gloss mapping
label_to_gloss = {}
with open(labels_file_path, mode='r', encoding='utf-8') as labels_file:
    csv_reader = csv.DictReader(labels_file)
    for row in csv_reader:
        label = int(row['id_label_in_documents'])
        gloss = row['name']
        label_to_gloss[label] = gloss

# Write video names, labels, and glosses to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['file', 'label', 'gloss'])

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv')):
            match = re.search(r'_(\d+)\.', filename)
            if match:
                label = int(match.group(1))
                gloss = label_to_gloss.get(label, 'Unknown')
            else:
                label = 'N/A'
                gloss = 'Unknown'

            full_filename = os.path.join(folder_path, filename)
            csv_writer.writerow([full_filename, label, gloss])

print(f'Video names have been written to {csv_file_path}')

# Find min label
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    labels = [int(row["label"]) for row in csv_reader if row["label"].isdigit()]  # Convert to int and filter out 'N/A'
    min_label = min(labels) if labels else None

print("Minimum label:", min_label)

# Normalize labels
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file, \
     open(final_file_path, mode='w', newline='', encoding='utf-8') as final_file:
    
    csv_reader = csv.DictReader(csv_file)
    fieldnames = csv_reader.fieldnames
    
    csv_writer = csv.DictWriter(final_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for row in csv_reader:
        if row['label'].isdigit():  # Check if label is a digit before converting
            row['label'] = str(int(row['label']) - min_label)  # Normalize and convert back to string
        csv_writer.writerow(row)

# Replace the original file with the updated file
os.replace(final_file_path, csv_file_path)

print("Labels have been updated and saved.")


In [6]:
%cd /home/ibmelab/Documents/GG/VSLRecognition/vsl/AAGCN/

/home/ibmelab/Documents/GG/VSLRecognition/vsl/AAGCN


In [ ]:
import pandas as pd
import random
import re

videos_df = pd.read_csv('videos_list.csv')
videos_df['label'] = videos_df['file'].apply(lambda x: int(x.split('_')[-1].split('.')[0]))

def extract_actor_name(file_name):
    parts = file_name.split('_')
    if len(parts) > 1:
        return '_'.join(parts[:2])
    return file_name.strip()

videos_df['actor'] = videos_df['file'].apply(lambda x: extract_actor_name(x.split('\\')[-1]))

#print(videos_df['actor'])

unique_actors = videos_df['actor'].unique()
train_actors = random.sample(list(unique_actors), 24)
remaining_actors = [actor for actor in unique_actors if actor not in train_actors]
valid_actors = random.sample(remaining_actors, 2)
test_actors = [actor for actor in remaining_actors if actor not in valid_actors]

train_videos = videos_df[videos_df['actor'].isin(train_actors)][['file', 'label']]
valid_videos = videos_df[videos_df['actor'].isin(valid_actors)][['file', 'label']]
test_videos = videos_df[videos_df['actor'].isin(test_actors)][['file', 'label']]

train_videos.to_csv('train.csv', index=False)
valid_videos.to_csv('valid.csv', index=False)
test_videos.to_csv('test.csv', index=False)

unique_actors.sort()
#print(unique_actors)

print(f'Train videos: {len(train_videos)}')
print(f'Validation videos: {len(valid_videos)}')
print(f'Test videos: {len(test_videos)}')


In [ ]:
import pandas as pd
import mediapipe as mp
import cv2
import os
from collections import defaultdict
from joblib import Parallel, delayed
from tqdm import tqdm

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

hand_landmarks = ['INDEX_FINGER_DIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_TIP', 
                  'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_TIP', 
                  'PINKY_DIP', 'PINKY_MCP', 'PINKY_PIP', 'PINKY_TIP', 'RING_FINGER_DIP', 'RING_FINGER_MCP', 
                  'RING_FINGER_PIP', 'RING_FINGER_TIP', 'THUMB_CMC', 'THUMB_IP', 'THUMB_MCP', 'THUMB_TIP', 'WRIST']
pose_landmarks = ['LEFT_ANKLE', 'LEFT_EAR', 'LEFT_ELBOW', 'LEFT_EYE', 'LEFT_EYE_INNER', 'LEFT_EYE_OUTER', 
                  'LEFT_FOOT_INDEX', 'LEFT_HEEL', 'LEFT_HIP', 'LEFT_INDEX', 'LEFT_KNEE', 'LEFT_PINKY', 
                  'LEFT_SHOULDER', 'LEFT_THUMB', 'LEFT_WRIST', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'NOSE', 
                  'RIGHT_ANKLE', 'RIGHT_EAR', 'RIGHT_ELBOW', 'RIGHT_EYE', 'RIGHT_EYE_INNER', 'RIGHT_EYE_OUTER', 
                  'RIGHT_FOOT_INDEX', 'RIGHT_HEEL', 'RIGHT_HIP', 'RIGHT_INDEX', 'RIGHT_KNEE', 'RIGHT_PINKY', 
                  'RIGHT_SHOULDER', 'RIGHT_THUMB', 'RIGHT_WRIST']

def extract_keypoint(video_path, label):
    video =   f"/home/ibmelab/Documents/GG/VSLRecognition/vsl/videos/{video_path}"
    cap = cv2.VideoCapture(video)
    
    keypoint_dict = defaultdict(list)
    count = 0

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            count += 1
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)

            if results.right_hand_landmarks:
                for idx, landmark in enumerate(results.right_hand_landmarks.landmark): 
                    keypoint_dict[f"{hand_landmarks[idx]}_right_x"].append(landmark.x)
                    keypoint_dict[f"{hand_landmarks[idx]}_right_y"].append(landmark.y)
                    keypoint_dict[f"{hand_landmarks[idx]}_right_z"].append(landmark.z)
            else:
                for idx in range(len(hand_landmarks)):
                    keypoint_dict[f"{hand_landmarks[idx]}_right_x"].append(0)
                    keypoint_dict[f"{hand_landmarks[idx]}_right_y"].append(0)
                    keypoint_dict[f"{hand_landmarks[idx]}_right_z"].append(0)

            if results.left_hand_landmarks:
                for idx, landmark in enumerate(results.left_hand_landmarks.landmark): 
                    keypoint_dict[f"{hand_landmarks[idx]}_left_x"].append(landmark.x)
                    keypoint_dict[f"{hand_landmarks[idx]}_left_y"].append(landmark.y)
                    keypoint_dict[f"{hand_landmarks[idx]}_left_z"].append(landmark.z)
            else:
                for idx in range(len(hand_landmarks)):
                    keypoint_dict[f"{hand_landmarks[idx]}_left_x"].append(0)
                    keypoint_dict[f"{hand_landmarks[idx]}_left_y"].append(0)
                    keypoint_dict[f"{hand_landmarks[idx]}_left_z"].append(0)

            if results.pose_landmarks:
                for idx, landmark in enumerate(results.pose_landmarks.landmark): 
                    keypoint_dict[f"{pose_landmarks[idx]}_x"].append(landmark.x)
                    keypoint_dict[f"{pose_landmarks[idx]}_y"].append(landmark.y)
                    keypoint_dict[f"{pose_landmarks[idx]}_z"].append(landmark.z)
            else:
                for idx in range(len(pose_landmarks)):
                    keypoint_dict[f"{pose_landmarks[idx]}_x"].append(0)
                    keypoint_dict[f"{pose_landmarks[idx]}_y"].append(0)
                    keypoint_dict[f"{pose_landmarks[idx]}_z"].append(0)

        keypoint_dict["frame"] = count
        keypoint_dict["video_path"] = video_path
        keypoint_dict["label"] = label

        return keypoint_dict

def process_videos(mode):
    csv_file = f"/home/ibmelab/Documents/GG/VSLRecognition/vsl/label1-200/label/labelThreeView/labelThreeViewTesting/{mode}_labels.csv"
    data = pd.read_csv(csv_file)

    keypoints_list = Parallel(n_jobs=-1)(  # Chạy song song với số lượng core tối đa
        delayed(extract_keypoint)(row['file_name'], row['label_id']) for index, row in data.iterrows()
    )

    # Tạo DataFrame và lưu vào CSV
    keypoints_df = pd.DataFrame(keypoints_list)
    keypoints_df.to_csv(f"{mode}_3views_set.csv", index=False)

if __name__ == '__main__':
    modes = ["train", "val", "test"]

    for mode in modes:
        process_videos(mode)


Try something new

In [ ]:
%cd /home/ibmelab/Documents/GG/VSLRecognition/vsl
import pandas as pd
import ast
import numpy as np
import os
from tqdm import tqdm
import math

def find_index(array):
    for i, num in enumerate(array):
        if num != 0:
            return i
    return -1  # Return -1 if no non-zero element is found

def curl_skeleton(array):
    if sum(array) == 0:
        return array
    for i, location in enumerate(array):
        if location != 0:
            continue
        else:
            if i == 0 or i == len(array) - 1:
                continue
            else:
                if array[i + 1] != 0:
                    array[i] = float((array[i - 1] + array[i + 1]) / 2)
                else:
                    j = find_index(array[i + 1:])
                    if j == -1:
                        continue
                    array[i] = float(((1 + j) * array[i - 1] + array[i + 1 + j]) / (2 + j))
    return array

def impute_missing_keypoints(poses):
    """Replace missing keypoints (zeros) with values from neighboring frames."""
    poses_imputed = poses.copy()
    num_frames, num_keypoints, _ = poses.shape
    for kpi in range(num_keypoints):
        # Find frames where keypoint is missing
        missing = np.where(np.all(poses[:, kpi, :] == 0, axis=1))[0]
        for idx in missing:
            # Find the nearest frame where the keypoint is not missing
            non_missing = np.where(np.any(poses[:, kpi, :] != 0, axis=1))[0]
            if len(non_missing) == 0:
                continue
            nearest = non_missing[np.argmin(np.abs(non_missing - idx))]
            poses_imputed[idx, kpi, :] = poses[nearest, kpi, :]
    return poses_imputed

def normalize_keypoints(poses):
    """Normalize keypoints to be in the range [0,1]."""
    min_vals = poses.min(axis=(0,1))
    max_vals = poses.max(axis=(0,1))
    range_vals = max_vals - min_vals
    range_vals[range_vals == 0] = 1  # Avoid division by zero
    poses_normalized = (poses - min_vals) / range_vals
    return poses_normalized

def calc_pose_flow(prev, current):
    """Calculate the flow (angle and magnitude) between two frames."""
    result = np.zeros_like(prev)  # Shape: (num_keypoints, 2)
    for kpi in range(prev.shape[0]):
        if np.all(prev[kpi] == 0) or np.all(current[kpi] == 0):
            result[kpi] = 0.0
            continue

        dx = current[kpi, 0] - prev[kpi, 0]
        dy = current[kpi, 1] - prev[kpi, 1]
        ang = math.atan2(dy, dx)
        mag = np.hypot(dx, dy)

        result[kpi, 0] = ang
        result[kpi, 1] = mag

    return result  # Shape: (num_keypoints, 2)

if __name__ == "__main__":
    hand_landmarks = [
        'INDEX_FINGER_DIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_TIP',
        'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_TIP',
        'PINKY_DIP', 'PINKY_MCP', 'PINKY_PIP', 'PINKY_TIP',
        'RING_FINGER_DIP', 'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_TIP',
        'THUMB_CMC', 'THUMB_IP', 'THUMB_MCP', 'THUMB_TIP', 'WRIST'
    ]

    HAND_IDENTIFIERS = [id + "_right" for id in hand_landmarks] + [id + "_left" for id in hand_landmarks]
    POSE_IDENTIFIERS = ["RIGHT_SHOULDER", "LEFT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW"]
    body_identifiers = HAND_IDENTIFIERS + POSE_IDENTIFIERS

    modes = ["train", "val", "test"]

    output_folder = "gcn_keypoints_v2"
    os.makedirs(output_folder, exist_ok=True)

    for mode in modes:
        print(f"Processing {mode}_3views_set.csv")
        dataset = pd.read_csv(f"AAGCN/{mode}_3views_set.csv")
        print(f"Number of videos in {mode} set: {len(dataset)}")

        for video_index, video in tqdm(dataset.iterrows(), total=dataset.shape[0]):
            video_name = video["video_path"]  # Assuming there's a 'video_path' column
            video_base = os.path.splitext(os.path.basename(video_name))[0]  # Get the base name of the video

            T = len(ast.literal_eval(video["INDEX_FINGER_DIP_right_x"]))
            num_keypoints = len(body_identifiers)
            keypoints_all_frames = np.zeros((T, num_keypoints, 2))

            for index, identifier in enumerate(body_identifiers):
                data_keypoint_preprocess_x = curl_skeleton(ast.literal_eval(video[identifier + "_x"]))
                data_keypoint_preprocess_y = curl_skeleton(ast.literal_eval(video[identifier + "_y"]))
                keypoints_all_frames[:, index, 0] = np.asarray(data_keypoint_preprocess_x)
                keypoints_all_frames[:, index, 1] = np.asarray(data_keypoint_preprocess_y)

            # Impute missing keypoints
            keypoints_all_frames = impute_missing_keypoints(keypoints_all_frames)

            # Normalize keypoints
            keypoints_all_frames = normalize_keypoints(keypoints_all_frames)

            # Create output folder for the video
            video_output_folder = os.path.join(output_folder, video_base)
            os.makedirs(video_output_folder, exist_ok=True)

            # Compute flow and save per frame
            prev_frame = keypoints_all_frames[0]
            for idx in range(1, len(keypoints_all_frames)):
                current_frame = keypoints_all_frames[idx]
                flow = calc_pose_flow(prev_frame, current_frame)
                # Save the flow data per frame
                output_file = os.path.join(video_output_folder, f"hand_flow_{(idx-1):05d}.npy")
                np.save(output_file, flow)
                prev_frame = current_frame

        print(f"Processing of {mode} set completed.")


End of trying

In [ ]:
import pandas as pd
import ast
import numpy as np
from tqdm import tqdm

def find_index(array):
    for i, num in enumerate(array):
        if num != 0:
            return i

def curl_skeleton(array):
    if sum(array) == 0:
        return array
    for i, location in enumerate(array):
        if location != 0:
            continue
        else:
            if i == 0 or i == len(array) - 1:
                continue
            else:
                if array[i + 1] != 0:
                    array[i] = float((array[i - 1] + array[i + 1]) / 2)
                else:
                    if sum(array[i:]) == 0:
                        continue
                    else:
                        j = find_index(array[i + 1:])
                        array[i] = float(((1 + j) * array[i - 1] + 1 * array[i + 1 + j]) / (2 + j))
    return array

if __name__ == "__main__":
    hand_landmarks = [
        'INDEX_FINGER_DIP', 'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_TIP', 
        'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_TIP', 
        'PINKY_DIP', 'PINKY_MCP', 'PINKY_PIP', 'PINKY_TIP', 
        'RING_FINGER_DIP', 'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_TIP', 
        'THUMB_CMC', 'THUMB_IP', 'THUMB_MCP', 'THUMB_TIP', 'WRIST'
    ]
    
    HAND_IDENTIFIERS = [id + "_right" for id in hand_landmarks] + [id + "_left" for id in hand_landmarks]
    POSE_IDENTIFIERS = ["RIGHT_SHOULDER", "LEFT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW"]
    body_identifiers = HAND_IDENTIFIERS + POSE_IDENTIFIERS 
    
    frames = 80
    modes = ["train", "val", "test"]
    
    for mode in modes:
        print(f"Processing {mode}_set.csv")
        train_data = pd.read_csv(f"{mode}_set.csv")
        print(len(train_data))
        
        data = []
        labels = []
        
        for video_index, video in tqdm(train_data.iterrows(), total=train_data.shape[0]):  # Ensure tqdm knows total count
            # Remove the print statement for row_index
            # row_index = video["video_path"]
            # print(row_index)

            T = len(ast.literal_eval(video["INDEX_FINGER_DIP_right_x"]))
            current_row = np.empty(shape=(2, T, len(body_identifiers), 1))
            for index, identifier in enumerate(body_identifiers):
                data_keypoint_preprocess_x = curl_skeleton(ast.literal_eval(video[identifier + "_x"]))
                current_row[0, :, index, :] = np.asarray(data_keypoint_preprocess_x).reshape(T, 1)
                data_keypoint_preprocess_y = curl_skeleton(ast.literal_eval(video[identifier + "_y"]))
                current_row[1, :, index, :] = np.asarray(data_keypoint_preprocess_y).reshape(T, 1)

            if T < frames:
                target = np.zeros(shape=(2, frames, len(body_identifiers), 1))
                target[:, :T, :, :] = current_row
            else:
                target = current_row[:, :frames, :, :]
                
            data.append(target)
            labels.append(int(video["label"]))

        keypoint_data = np.stack(data, axis=0)
        label_data = np.stack(labels, axis=0)
        np.save(f'vsl199_{mode}_data_preprocess.npy', keypoint_data)
        np.save(f'vsl199_{mode}_label_preprocess.npy', label_data)
        print("Processed and saved successfully.")


In [4]:
import numpy as np

data = np.load('vsl199_test_label_preprocess.npy')
data_long = data.astype('int64')
np.save('test_label_preprocess.npy', data_long)

data = np.load('vsl199_train_label_preprocess.npy')
data_long = data.astype('int64')
np.save('train_label_preprocess.npy', data_long)

data = np.load('vsl199_val_label_preprocess.npy')
data_long = data.astype('int64')
np.save('val_label_preprocess.npy', data_long)

In [ ]:
%cd /home/ibmelab/Documents/GG/VSLRecognition/AUTSL/AAGCN
import pandas as pd
df = pd.read_csv('full_labels.csv')
unique_labels = df['label'].unique()
num_labels = len(unique_labels)
print(num_labels)

unique_labels.sort()
print(unique_labels)

In [ ]:
%cd /home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN
from torch.utils.data import DataLoader
import torch
from torchinfo import summary
from feeder import FeederINCLUDE
from aagcn import AAGCN
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
from augumentation import Rotate, Left, Right, GaussianNoise, Compose
from torch.utils.data import random_split
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

if __name__ == '__main__':

    # Hyper parameter tuning : batch_size, learning_rate, weight_decay
    #batch_size: 2->1
    config = {'batch_size': 170, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}
    # Load device
    device = "cuda:1" if torch.cuda.is_available() else "cpu"
    # Load model

    # num_class: 101 -> 3 (= number of labels)
    model = AAGCN(num_class=num_labels, num_point=46, num_person=1, in_channels=2,
                graph_args = {"layout" :"mediapipe_two_hand", "strategy": "spatial"},
                learning_rate=config["learning_rate"], weight_decay=config["weight_decay"])

    # Callback PL
    callbacks = [
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="valid_loss",
            mode="min",
            every_n_epochs = 2,
            filename='{epoch}-{valid_accuracy:.2f}-autsl-aagcn-smaller-model'
        ),
    ]
    # Augument 
    batch_size = config["batch_size"]
    transforms = Compose([
        Rotate(15, 80, 25, (0.5, 0.5))
    ])

    # Dataset class
    ''' Đổi tên path
    train_dataset = FeederINCLUDE(data_path=f"wsl100_train_data_preprocess.npy", label_path=f"wsl100_train_label_preprocess.npy",
                            transform=transforms)
    test_dataset = FeederINCLUDE(data_path=f"wsl100_test_data_preprocess.npy", label_path=f"wsl100_test_label_preprocess.npy")
    valid_dataset = FeederINCLUDE(data_path=f"wsl100_valid_data_preprocess.npy", label_path=f"wsl100_valid_label_preprocess.npy")
    '''
    %cd /home/ibmelab/Documents/GG/VSLRecognition/AUTSL/AAGCN/
    train_dataset = FeederINCLUDE(data_path=f"autsl_train_data_preprocess.npy", label_path=f"train_label_preprocess.npy",
                            transform=transforms)
    test_dataset = FeederINCLUDE(data_path=f"autsl_test_data_preprocess.npy", label_path=f"test_label_preprocess.npy")
    valid_dataset = FeederINCLUDE(data_path=f"autsl_valid_data_preprocess.npy", label_path=f"valid_label_preprocess.npy")

    # DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)
    val_dataloader = DataLoader(valid_dataset, batch_size=config["batch_size"], shuffle=False)

    specific_batch = next(iter(train_dataloader))
    print("Input shape ", specific_batch[0].shape)
    print("Data loader success")
    # Trainer PL
    %cd /home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN
    trainer = pl.Trainer(max_epochs = 120, accelerator="auto", check_val_every_n_epoch = 1, 
                       devices = 1, callbacks=callbacks)
                    #  , logger=wandb_logger) # wandb
    trainer.fit(model, train_dataloader, val_dataloader)
    # Test PL (When test find the right ckpt_path and comment code line 58)
    # trainer.test(model, test_dataloader, ckpt_path="checkpoints/epoch=61-valid_accuracy=0.91-vsl_100-aagcn-2hand+preprocessing_keypoint+augment(v1).ckpt", 
                # verbose=True)

In [5]:
import numpy as np

# Load the NumPy file
# array = np.load('/home/ibmelab/Documents/GG/VSLRecognition/vsl/AAGCN/vsl199_train_data_preprocess.npy')
# array = np.load('/home/ibmelab/Documents/GG/VSLRecognition/vsl/gcn_keypoints_v2/01_Co-Hien_1-100_1-2-3_0108___center_device02_signer01_center_ord1_2/hand_flow_00000.npy')
array = np.load('/home/ibmelab/Documents/GG/VSLRecognition/vsl/poseflow/01_Co-Hien_1-100_1-2-3_0108___center_device02_signer01_center_ord1_2/flow_00007.npy')
# Print the shape of the array
print(array.shape)


(135, 2)


In [3]:
%cd /home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN/
import torch
from torch.utils.data import DataLoader
from feeder import FeederINCLUDE, FeederCustomV2
from aagcn import AAGCN
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from augumentation import Rotate, Compose, GaussianNoise
from pytorch_lightning.utilities.migration import pl_legacy_patch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
if __name__ == '__main__':
    # Hyperparameters
    config = {'batch_size': 140, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}
    num_labels = 199  # Set your new number of classes here

    # Load device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Initialize the model with the new number of classes
    model = AAGCN(
        num_class=num_labels,
        num_point=46,
        num_person=1,
        in_channels=2,
        graph_args={"layout": "mediapipe_two_hand", "strategy": "spatial"},
        learning_rate=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    # Path to your checkpoint
    checkpoint_path = "checkpoints/epoch=37-valid_accuracy=0.88-autsl-aagcn-smaller-model.ckpt"

    # Load the checkpoint
    with pl_legacy_patch():
        checkpoint = torch.load(checkpoint_path, map_location=device)

    # Get the state dict
    state_dict = checkpoint['state_dict']

    # Remove the keys for the final layer (adjust 'fc' to match your model's final layer name)
    filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('fc.')}

    # Load the filtered state dict into the model
    model.load_state_dict(filtered_state_dict, strict=False)

    # Callbacks
    callbacks = [
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="valid_loss",
            mode="min",
            every_n_epochs=2,
            filename='{epoch}-{valid_accuracy:.2f}-vsl199-small-model'
        ),
    ]

    batch_size = config["batch_size"]
    transforms = Compose([
        Rotate(15, 80, 25, (0.5, 0.5)),
        GaussianNoise(),
    ])
    

    %cd /home/ibmelab/Documents/GG/VSLRecognition/vsl/AAGCN/
    # Datasets
    train_dataset = FeederINCLUDE(
        data_path="vsl199_train_data_preprocess.npy",
        label_path="train_label_preprocess.npy",
        transform=transforms
    )
    test_dataset = FeederINCLUDE(
        data_path="vsl199_test_data_preprocess.npy",
        label_path="test_label_preprocess.npy"
    )
    valid_dataset = FeederINCLUDE(
        data_path="vsl199_val_data_preprocess.npy",
        label_path="val_label_preprocess.npy"
    )
    # def gcn_bert_collate_fn_(batch):
    #     labels = torch.stack([s[1] for s in batch],dim = 0)
    #     keypoints = torch.stack([s[0] for s in batch],dim = 0) # bs t n c
                                                                                                             
    #     return {'keypoints':keypoints},labels

    # collate_func = gcn_bert_collate_fn_
    # train_dataset = FeederCustomV2('/home/ibmelab/Documents/GG/VSLRecognition/vsl','train')
    # test_dataset = FeederCustomV2('/home/ibmelab/Documents/GG/VSLRecognition/vsl','test')
    # valid_dataset = FeederCustomV2('/home/ibmelab/Documents/GG/VSLRecognition/vsl','val')

    # DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True,
                                  num_workers = 12, prefetch_factor = 4, persistent_workers =  True)
    test_dataloader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False,
                                 num_workers = 12, prefetch_factor = 4, persistent_workers =  True)
    val_dataloader = DataLoader(valid_dataset, batch_size=config["batch_size"], shuffle=False,
                                num_workers = 12, prefetch_factor = 4, persistent_workers =  True)

    %cd /home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN/
    # Trainer
    trainer = pl.Trainer(
        max_epochs=120,
        accelerator="auto",
        check_val_every_n_epoch=1,
        devices=1,
        callbacks=callbacks
    )

    # Start training
    trainer.fit(model, train_dataloader, val_dataloader)


/home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (41, 37), (37, 39), (39, 38), (38, 40), (41, 22), (22, 23), (23, 21), (21, 24), (22, 26), (26, 27), (27, 25), (25, 28), (26, 34), (34, 35), (35, 33), (33, 36), (41, 30), (30, 31), (31, 29), (29, 32), (20, 16), (16, 18), (18, 17), (17, 19), (20, 1), (1, 2), (2, 0), (0, 3), (1, 5), (5, 6), (6, 4), (4, 7), (5, 13), (13, 14), (14, 12), (12, 15), (20, 9), (9, 10), (10, 8), (8, 11), (42, 45), (45, 20), (42, 43), (43, 44), (44, 41)]
/home/ibmelab/Documents/GG/VSLRecognition/vsl/AAGCN
/home/ibmela

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-10-10 16:04:00.538382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 16:04:00.556358: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 16:04:00.561740: E external/local_xla/xla/stream_exe

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/ibmelab/anaconda3/envs/son/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/ibmelab/anaconda3/envs/son/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (31) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=120` reached.
/home/ibmelab/anaconda3/envs/son/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
%cd /home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN/
import torch
from torch.utils.data import DataLoader
from feeder import FeederINCLUDE, FeederCustomV2
from aagcn import AAGCN
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from augumentation import Rotate, Compose
from pytorch_lightning.utilities.migration import pl_legacy_patch


if __name__ == '__main__':
    # Hyperparameters
    config = {'batch_size': 90, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}
    num_labels = 199  # Set your new number of classes here

    # Load device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Initialize the model with the new number of classes
    model = AAGCN(
        num_class=num_labels,
        num_point=46,
        num_person=1,
        in_channels=2,
        graph_args={"layout": "mediapipe_two_hand", "strategy": "spatial"},
        learning_rate=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    # Path to your checkpoint
    checkpoint_path = "checkpoints/epoch=95-valid_accuracy=0.73-vsl199.ckpt"

    # Load the checkpoint
    with pl_legacy_patch():
        checkpoint = torch.load(checkpoint_path, map_location=device)

    # Get the state dict
    state_dict = checkpoint['state_dict']

    # Remove the keys for the final layer (adjust 'fc' to match your model's final layer name)
    # filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('fc.')}

    # Load the filtered state dict into the model
    model.load_state_dict(state_dict, strict=False)

    # Callbacks
    callbacks = [
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="valid_loss",
            mode="min",
            every_n_epochs=2,
            filename='{epoch}-{valid_accuracy:.2f}-vsl199-FeederCustom'
        ),
    ]

    batch_size = config["batch_size"]
    transforms = Compose([
        Rotate(15, 80, 25, (0.5, 0.5))
    ])
    

    %cd /home/ibmelab/Documents/GG/VSLRecognition/vsl/AAGCN/
    # Datasets
    train_dataset = FeederINCLUDE(
        data_path="vsl199_train_data_preprocess.npy",
        label_path="train_label_preprocess.npy",
        transform=transforms
    )
    test_dataset = FeederINCLUDE(
        data_path="vsl199_test_data_preprocess.npy",
        label_path="test_label_preprocess.npy"
    )
    valid_dataset = FeederINCLUDE(
        data_path="vsl199_val_data_preprocess.npy",
        label_path="val_label_preprocess.npy"
    )
    # def gcn_bert_collate_fn_(batch):
    #     labels = torch.stack([s[1] for s in batch],dim = 0)
    #     keypoints = torch.stack([s[0] for s in batch],dim = 0) # bs t n c
                                                                                                             
    #     return {'keypoints':keypoints},labels

    # collate_func = gcn_bert_collate_fn_
    # train_dataset = FeederCustomV2('/home/ibmelab/Documents/GG/VSLRecognition/vsl','train')
    # test_dataset = FeederCustomV2('/home/ibmelab/Documents/GG/VSLRecognition/vsl','test')
    # valid_dataset = FeederCustomV2('/home/ibmelab/Documents/GG/VSLRecognition/vsl','val')

    # DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True,
                                  num_workers = 12, prefetch_factor = 4, persistent_workers =  True)
    test_dataloader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False,
                                 num_workers = 12, prefetch_factor = 4, persistent_workers =  True)
    val_dataloader = DataLoader(valid_dataset, batch_size=config["batch_size"], shuffle=False,
                                num_workers = 12, prefetch_factor = 4, persistent_workers =  True)

    %cd /home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN/
    # Trainer
    trainer = pl.Trainer(
        max_epochs=120,
        accelerator="auto",
        check_val_every_n_epoch=1,
        devices=1,
        callbacks=callbacks
    )

    # Start training
    trainer.fit(model, train_dataloader, val_dataloader)


In [4]:
%cd /home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN
from torch.utils.data import DataLoader
import torch
from torchinfo import summary
from feeder import FeederINCLUDE
from aagcn import AAGCN
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
from augumentation import Rotate, Left, Right, GaussianNoise, Compose
from torch.utils.data import random_split

if __name__ == '__main__':

    # Hyper parameter tuning : batch_size, learning_rate, weight_decay
    #batch_size: 2->1
    config = {'batch_size': 170, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}
    # Load device
    device = "cuda:1" if torch.cuda.is_available() else "cpu"
    # Load model

    # num_class: 101 -> 3 (= number of labels)
    model = AAGCN(num_class=199, num_point=46, num_person=1, in_channels=2,
                graph_args = {"layout" :"mediapipe_two_hand", "strategy": "spatial"},
                learning_rate=config["learning_rate"], weight_decay=config["weight_decay"])

    # Callback PL
    callbacks = [
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="valid_accuracy",
            mode="max",
            every_n_epochs = 2,
            filename='{epoch}-{valid_accuracy:.2f}-wsl_100-aagcn-{fold}'
        ),
    ]
    # Augument 
    batch_size = config["batch_size"]
    transforms = Compose([
        Rotate(15, 80, 25, (0.5, 0.5))
    ])

    # Dataset class
    ''' Đổi tên path
    train_dataset = FeederINCLUDE(data_path=f"wsl100_train_data_preprocess.npy", label_path=f"wsl100_train_label_preprocess.npy",
                            transform=transforms)
    test_dataset = FeederINCLUDE(data_path=f"wsl100_test_data_preprocess.npy", label_path=f"wsl100_test_label_preprocess.npy")
    valid_dataset = FeederINCLUDE(data_path=f"wsl100_valid_data_preprocess.npy", label_path=f"wsl100_valid_label_preprocess.npy")
    '''
    %cd /home/ibmelab/Documents/GG/VSLRecognition/vsl/AAGCN/
    train_dataset = FeederINCLUDE(data_path=f"vsl199_train_data_preprocess.npy", label_path=f"train_label_preprocess.npy",
                            transform=transforms)
    test_dataset = FeederINCLUDE(data_path=f"vsl199_test_data_preprocess.npy", label_path=f"test_label_preprocess.npy")
    valid_dataset = FeederINCLUDE(data_path=f"vsl199_val_data_preprocess.npy", label_path=f"val_label_preprocess.npy")

    # DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)
    val_dataloader = DataLoader(valid_dataset, batch_size=config["batch_size"], shuffle=False)

    specific_batch = next(iter(train_dataloader))
    print("Input shape ", specific_batch[0].shape)
    print("Data loader success")
    # Trainer PL
    %cd /home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN
    trainer = pl.Trainer(max_epochs = 120, accelerator="auto", check_val_every_n_epoch = 1, 
                       devices = 1, callbacks=callbacks)
                    #  , logger=wandb_logger) # wandb
    #trainer.fit(model, train_dataloader, val_dataloader)
    # Test PL (When test find the right ckpt_path and comment code line 58)
    trainer.test(model, test_dataloader, ckpt_path="checkpoints/epoch=15-valid_accuracy=0.70-vsl199-small-model.ckpt", 
                verbose=True)

/home/ibmelab/Documents/GG/VSLRecognition/HandSignRecogDev/AAGCN


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at checkpoints/epoch=15-valid_accuracy=0.70-vsl199-small-model.ckpt


[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (41, 37), (37, 39), (39, 38), (38, 40), (41, 22), (22, 23), (23, 21), (21, 24), (22, 26), (26, 27), (27, 25), (25, 28), (26, 34), (34, 35), (35, 33), (33, 36), (41, 30), (30, 31), (31, 29), (29, 32), (20, 16), (16, 18), (18, 17), (17, 19), (20, 1), (1, 2), (2, 0), (0, 3), (1, 5), (5, 6), (6, 4), (4, 7), (5, 13), (13, 14), (14, 12), (12, 15), (20, 9), (9, 10), (10, 8), (8, 11), (42, 45), (45, 20), (42, 43), (43, 44), (44, 41)]
/home/ibmelab/Documents/GG/VSLRecognition/vsl/AAGCN
Input shape  torch.Size([170, 2, 80, 46, 1])
Data loader success
/home/ibmela

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
Loaded model weights from the checkpoint at checkpoints/epoch=15-valid_accuracy=0.70-vsl199-small-model.ckpt
/home/ibmelab/anaconda3/envs/son/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

Targets :  tensor([  1,  10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,  11, 110,
        111, 112, 113, 114, 115, 116, 117, 118, 119,  12, 120, 121, 122, 123,
        124, 125, 126, 127, 128, 129,  13, 130, 131, 132, 133, 134, 135, 136,
        137, 138, 139,  14, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
         15, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,  16, 160, 161,
        162, 163, 164, 165, 166, 167, 168, 169,  17, 170, 171, 172, 173, 174,
        175, 176, 177, 178, 179,  18, 180, 181, 182, 183, 184, 185, 186, 187,
        188, 189,  19, 190, 191, 192, 193, 194, 195, 196, 197, 198,   2,  20,
         21,  22,  23,  24,  25,  26,  27,  28,  29,   3,  30,  31,  32,  33,
         34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,
         48,  49,   5,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,   6,
         60,  61,  62,  63,  64,  65,  66,  67,  68,  69,   7,  70,  71,  72,
         73,  74], device='cuda:0')
Preds :  tensor([

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.835220217704773     │
│         test_loss         │    0.5644031167030334     │
└───────────────────────────┴───────────────────────────┘